## Model Metrics Analysis for LLaMa-2 benchmarking
---------
*This notebook works best with the conda_python3 kernel on a ml.t3.medium machine*.

### This part of our solution design includes the chunk of taking the metrics generated and creating visualizations from it for further analysis to make decisions more quicker, efficient, and cost optimal.

- In this file, we will go over and create side by side visualizations of different models deployed, how their inference latency is impacted based on the concurrency level, instance size and different model configurations. Using these visualizations and charts, making executive decisions, saving on time and cost becomes critical. 


- In this notebook, we will also record the error rates for each of the deployed model endpoints based on how it ran against different metrics as specified above. These visualizations will be applicable and work for any and every jumpstart and non jumpstart model if deployed correctly using the prior steps.

#### Import all of the necessary libraries below to run this notebook

In [1]:
# if interactive mode is set to no -> pickup fmbench from Python installation path
# if interactive mode is set to yes -> pickup fmbench from the current path (one level above this notebook)
# if interactive mode is not defined -> pickup fmbench from the current path (one level above this notebook)
# the premise is that if run non-interactively then it can only be run through main.py which will set interactive mode to no
import os
import sys
if os.environ.get("INTERACTIVE_MODE_SET", "yes") == "yes":
    sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import io
import json
import datetime
import numpy as np
import pandas as pd

## Import seaborn and other related libraries for visualizations and plotting charts
import seaborn as sns
from pathlib import Path
from tomark import Tomark
from fmbench.utils import *
from fmbench.globals import *
from datetime import datetime
from datetime import timezone
from dateutil.parser import parse
from typing import List, Optional

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


config file current -> configs/config-llama2-7b-g4dn-g5-trt.yml, None
Loaded config: {'general': {'name': 'Llama2-7b-g4dn-g5', 'model_name': 'Llama2-7b'}, 'aws': {'region': 'us-east-1', 'sagemaker_execution_role': 'arn:aws:sts::015469603702:assumed-role/fmbench-role/SageMaker', 'bucket': 'sagemaker-fmbench-write-015469603702'}, 'dir_paths': {'data_prefix': 'data', 'prompts_prefix': 'prompts', 'all_prompts_file': 'all_prompts.csv', 'metrics_dir': 'metrics', 'models_dir': 'models', 'metadata_dir': 'metadata'}, 's3_read_data': {'read_bucket': 'sagemaker-fmbench-read-015469603702', 'scripts_prefix': 'scripts', 'script_files': ['hf_token.txt'], 'source_data_prefix': 'source_data', 'source_data_files': ['2wikimqa_e.jsonl', '2wikimqa.jsonl', 'hotpotqa_e.jsonl', 'hotpotqa.jsonl', 'narrativeqa.jsonl', 'triviaqa_e.jsonl', 'triviaqa.jsonl'], 'tokenizer_prefix': 'tokenizer', 'prompt_template_dir': 'prompt_template', 'prompt_template_file': 'prompt_template_llama2.txt'}, 'run_steps': {'0_setup.ipyn

In [3]:
## for plotting
import matplotlib.pyplot as plt
# rcParams for configuring Matplotlib settings
from matplotlib import rcParams

# figure size in inches
rcParams['figure.figsize'] = 10, 6

In [4]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Load the Config.yml file contains information that is used across this benchmarking environment, such as information about the aws account, prompts, payloads to be used for invocations

In [5]:
config = load_config(CONFIG_FILE)
logger.info(json.dumps(config, indent=2))

[2024-04-02 00:48:48,460] p26924 {635462509.py:2} INFO - {
  "general": {
    "name": "Llama2-7b-g4dn-g5",
    "model_name": "Llama2-7b"
  },
  "aws": {
    "region": "us-east-1",
    "sagemaker_execution_role": "arn:aws:sts::015469603702:assumed-role/fmbench-role/SageMaker",
    "bucket": "sagemaker-fmbench-write-015469603702"
  },
  "dir_paths": {
    "data_prefix": "data",
    "prompts_prefix": "prompts",
    "all_prompts_file": "all_prompts.csv",
    "metrics_dir": "metrics",
    "models_dir": "models",
    "metadata_dir": "metadata"
  },
  "s3_read_data": {
    "read_bucket": "sagemaker-fmbench-read-015469603702",
    "scripts_prefix": "scripts",
    "script_files": [
      "hf_token.txt"
    ],
    "source_data_prefix": "source_data",
    "source_data_files": [
      "2wikimqa_e.jsonl",
      "2wikimqa.jsonl",
      "hotpotqa_e.jsonl",
      "hotpotqa.jsonl",
      "narrativeqa.jsonl",
      "triviaqa_e.jsonl",
      "triviaqa.jsonl"
    ],
    "tokenizer_prefix": "tokenizer",
  

In [7]:
metrics_path_file: str = os.path.join(METADATA_DIR, METRICS_PATH_FNAME)
METRICS_DIR: str = Path(metrics_path_file).read_text().strip()
logger.info(f"metrics_path_file={metrics_path_file}, METRICS_DIR={METRICS_DIR}")

FileNotFoundError: [Errno 2] No such file or directory: 'metadata/metrics_path.txt'

In [ ]:
file_path = os.path.join(METRICS_DIR, config["report"]["per_inference_request_file"])
logger.info(f"File path containing the metrics per inference folder --> {file_path}")

# Read the file from S3
try:    
    file_content = get_s3_object(config['aws']['bucket'], file_path)
    # Use pandas to read the CSV content
    df_per_inference = pd.read_csv(io.StringIO(file_content))
    logger.info(f"{file_path} read into dataframe of shape {df_per_inference.shape}")
    df_per_inference.head()
except Exception as e:
    logger.error(f"Error reading from S3: {e}")

df_per_inference.head()

### Relationship between prompt token length and inference latency for different instances and concurrency levels

In [ ]:
# Rename a column in the dataframe for clarity of the instance parameter of the model used
df_per_inference = df_per_inference.rename(columns={"instance_type": "instance"})

if df_per_inference.latency.min() < 1:
    print("multiplying by 1000")
    latency_units = "milliseconds"
    multiplier = 1000
    step_size = 500
    df_per_inference.latency = df_per_inference.latency*1000
else:
    multiplier = 10
    step_size = 5
    latency_units = "seconds"

## Initializing yticks and title for the chart
yticks: Optional[List] = None
title: Optional[str] = None

if config['report'].get('latency_vs_token_len_chart'):
    yticks: List = config['report']['latency_vs_token_len_chart'].get('y_ticks')
    title: str = config['report']['latency_vs_token_len_chart'].get('title')

if title is None:
    title = "Effect of token length on inference latency"

if len(df_per_inference.instance.unique()) == 1:
    # This created a FacetGrid for plotting multiple scatter plots based on 'instance' and 'concurrency' categories
    g = sns.FacetGrid(df_per_inference, col="concurrency", hue="instance", height=3.5, aspect=1.25, col_wrap=3)
else: 
    g = sns.FacetGrid(df_per_inference, col="concurrency", row="instance", hue="instance", height=3.5, aspect=1.25)

## Subtitle of the facetgrid
g.fig.suptitle(title)

# # This will map a scatterplot to the FacetGrid for each subset of the data
sns_plot = g.map(sns.scatterplot, "prompt_tokens", "latency")

# Set the y-axis label for all plots
g = g.set_ylabels(f"Latency ({latency_units})")
    
if yticks is None:
    # Y-axis ticks based on the maximum latency value and setting them in that manner
    yticks: List = list(range(0, (int(df_per_inference.latency.max())//multiplier+2)*multiplier, step_size))

g = g.set(yticks=yticks)

# Set the x-axis label for all plots as the prompt length or tokens
g = g.set_xlabels("Prompt length (tokens)")

# Create a bytes buffer to save the plot
buffer = io.BytesIO()
sns_plot.savefig(buffer, format='png')
buffer.seek(0)  # Rewind buffer to the beginning

# Write the plot to S3
write_to_s3(buffer.getvalue(), BUCKET_NAME, "", METRICS_DIR, TOKENS_VS_LATENCY_PLOT_FNAME)
logger.info(f"Plot saved to s3://{BUCKET_NAME}/{METRICS_DIR}/{TOKENS_VS_LATENCY_PLOT_FNAME}")

# Optionally, display the plot
sns_plot

In [ ]:
df_per_inference.latency.describe()

In [ ]:
yticks

#### Define the all metrics file path and read it to generate visualizations

In [ ]:
all_metrics_fpath = os.path.join(METRICS_DIR, config["report"]["all_metrics_file"])

# Read the file from S3
try:
    file_content = get_s3_object(BUCKET_NAME, all_metrics_fpath)

    # Use pandas to read the CSV content
    df_all_metrics = pd.read_csv(io.StringIO(file_content))
    logger.info(f"{all_metrics_fpath} read into dataframe of shape {df_all_metrics.shape}")
    df_all_metrics.head()
except Exception as e:
    logger.error(f"Error reading from S3: {e}")

df_all_metrics.head()

In [ ]:
## displaying all of the available columns in the all metrics dataframe
df_all_metrics.columns

#### Display the number of experiment names within the metrics dataframe, instance types and models

In [ ]:
experiments = df_all_metrics.experiment_name.unique()
instance_types = df_all_metrics.instance_type.unique()
model_names = df_all_metrics.ModelName.unique()
logger.info(f"contains information about {len(experiments)} experiments, {len(instance_types)} instance types, {len(model_names)} models")

In [ ]:
## extract some of the columns
relevant_cols = ['experiment_name',
                   'payload_file',
                     'instance_type',
                       'concurrency',
                         'error_rate',
                           'prompt_token_count_mean',
                             'prompt_token_throughput',
                               'completion_token_count_mean',
                                 'completion_token_throughput',
                                   'latency_mean',
                                     'transactions_per_minute']

## initialize a group by columns to use further in generating portions of the dataframe and filtering it
group_by_cols = ['experiment_name',
                   'payload_file',
                     'instance_type',
                      'concurrency']


#### Create an 'experiment_counts.csv' to store metrics on experiment name, the payload file, concurrency and the total counts associated to that given experiment to visualize the distribution

In [ ]:
df_counts = df_all_metrics[group_by_cols].value_counts().reset_index()

# Convert df_counts to CSV format
csv_buffer = io.StringIO()
df_counts.to_csv(csv_buffer, index=False)
csv_data = csv_buffer.getvalue()

# Define the file name and the S3 path
COUNTS_FNAME = "experiment_counts.csv"
counts_s3_path = os.path.join(METRICS_DIR, COUNTS_FNAME)

# Write the CSV data to S3
write_to_s3(csv_data, BUCKET_NAME, "", METRICS_DIR, COUNTS_FNAME)
logger.info(f"Counts DataFrame saved to s3://{BUCKET_NAME}/{counts_s3_path}")

df_counts

#### Display the mean error rates for each experiment with different congifurations using the same columns of interest used in the cell above

In [ ]:
df_error_rates = df_all_metrics.groupby(group_by_cols).agg({'error_rate': 'mean'}).reset_index()
df_error_rates = df_error_rates.round(2)

csv_buffer = io.StringIO()
df_error_rates.to_csv(csv_buffer, index=False)
error_csv = csv_buffer.getvalue()

# Define the file name and the S3 path
ERROR_RATES_FNAME: str = "error_rates.csv"
counts_s3_path = os.path.join(METRICS_DIR, ERROR_RATES_FNAME)

# Write the CSV data to S3
write_to_s3(error_csv, BUCKET_NAME, "", METRICS_DIR, ERROR_RATES_FNAME)
logger.info(f"Error Counts DataFrame saved to s3://{BUCKET_NAME}/{counts_s3_path}")

df_error_rates

### Visualize Inference error rates across different concurrency levels and instance types

In [ ]:
df_error_rates = df_error_rates.rename(columns={"instance_type": "instance", "payload_file": "dataset"})

# Clean up the dataset names by removing json file extensions and prefixes
df_error_rates.dataset = df_error_rates.dataset.map(lambda x: x.replace(".jsonl", "").replace("payload_", ""))

# this creates a facetGrid for plotting scatter plots based on 'instance' and 'dataset'
g = sns.FacetGrid(df_error_rates, col="instance", row="dataset", hue="instance", height=3.5, aspect=1.25)

# Maps a scatterplot to the FacetGrid for each subset of the data
sns_plot = g.map(sns.scatterplot, "concurrency", "error_rate")

# Create a subtitle
g.fig.suptitle("Inference error rates for different concurrency levels and instance types")

## Set x and y labels for this chart
g = g.set_ylabels("Error rate (failed / total inferences)")
g = g.set_xlabels("Concurrency level")

sns_plot.savefig(buffer, format='png')
buffer.seek(0)

# Write the plot to S3
write_to_s3(buffer.getvalue(), BUCKET_NAME, "", METRICS_DIR, ERROR_RATES_PLOT_FNAME)
logger.info(f"Plot saved to s3://{BUCKET_NAME}/{METRICS_DIR}/{ERROR_RATES_PLOT_FNAME}")

## Display the plot 
sns_plot

#### Check for the df elements that have error rates above 0

In [ ]:
df_error_rates_nz = df_error_rates[df_error_rates.error_rate > 0]
df_error_rates_nz

In [ ]:
## initialize a dataframe to get the mean of the columns in consideration
df_summary_metrics = df_all_metrics[relevant_cols].groupby(group_by_cols).mean().reset_index()

# ugly way of doing this, will refactor this later (maybe)
df_summary_metrics.fillna(PLACE_HOLDER, inplace=True)
int_cols = ['prompt_token_count_mean', 'prompt_token_throughput', 'completion_token_count_mean', 'completion_token_throughput', 'transactions_per_minute']
for ic in int_cols:
    df_summary_metrics[ic] = df_summary_metrics[ic].astype(int)

df_summary_metrics.replace(PLACE_HOLDER, np.nan, inplace=True)
df_summary_metrics.latency_mean	= df_summary_metrics.latency_mean.round(2)
df_summary_metrics.error_rate	= df_summary_metrics.error_rate.round(2)

csv_buffer = io.StringIO()
df_summary_metrics.to_csv(csv_buffer, index=False)
summary_metrics_csv = csv_buffer.getvalue()

# Define the file name for S3 based on the original file path
summary_file_name = all_metrics_fpath.replace("all_metrics", "all_metrics_summary").split('/')[-1] 
summary_s3_path = os.path.join(METRICS_DIR, summary_file_name)  # Define full S3 path

# Write the CSV data to S3
write_to_s3(summary_metrics_csv, BUCKET_NAME, "", METRICS_DIR, summary_file_name)
logger.info(f"Summary metrics DataFrame saved to s3://{BUCKET_NAME}/{summary_s3_path}")

df_summary_metrics

In [ ]:
df_summary_metrics_nz = df_summary_metrics[df_summary_metrics.error_rate == 0]
logger.info(f"there are {len(df_summary_metrics_nz)} entries out of {len(df_summary_metrics)} in the summary data for which error rate is 0")

In [ ]:
df_summary_metrics_dataset = df_summary_metrics[df_summary_metrics.payload_file.str.contains(config['metrics']['dataset_of_interest'])]
logger.info(f"shape of dataframe with summary metrics for {config['metrics']['dataset_of_interest']} is {df_summary_metrics_dataset.shape}")

csv_buffer = io.StringIO()
df_summary_metrics_dataset.to_csv(csv_buffer, index=False)
metrics_dataset = csv_buffer.getvalue()

# Write the CSV data to S3
write_to_s3(metrics_dataset, BUCKET_NAME, "", METRICS_DIR, SUMMARY_METRICS_W_PRICING_FNAME)
logger.info(f"Summary metrics dataset saved to s3://{BUCKET_NAME}/{METRICS_DIR}/{SUMMARY_METRICS_W_PRICING_FNAME}")

df_summary_metrics_dataset

In [ ]:
df_all_metrics_for_dataset = df_all_metrics.rename(columns={"instance_type": "instance", "payload_file": "dataset"})
df_all_metrics_for_dataset.dataset = df_all_metrics_for_dataset.dataset.map(lambda x: x.replace(".jsonl", "").replace("payload_", ""))
ds = config['metrics']['dataset_of_interest']
df_all_metrics_for_dataset = df_all_metrics_for_dataset[df_all_metrics_for_dataset.dataset.str.contains(ds)]
# df_all_metrics_for_dataset.concurrency = df_all_metrics_for_dataset.concurrency.astype(str)
row_order = list(df_all_metrics_for_dataset[["instance", "latency_mean"]].groupby("instance").mean("latency_mean").reset_index()["instance"])
print(row_order)
sns_plot = sns.catplot(
    data=df_all_metrics_for_dataset, x='concurrency', y='latency_mean',
    col='instance', kind='box', col_wrap=len(row_order), hue="instance", row_order=row_order #, height=4.5, aspect=1.25
)
sns_plot._legend.remove()
sns_plot.fig.suptitle(f"Effect of concurrency on inference latency for each instance type for the {ds} dataset\n\n")
sns_plot = sns_plot.set_ylabels("Latency (seconds)")
sns_plot = sns_plot.set_xlabels("Concurrency level")
sns_plot.fig.subplots_adjust(top=0.8)

sns_plot.savefig(buffer, format='png')
buffer.seek(0)

# Write the plot to S3
write_to_s3(buffer.getvalue(), BUCKET_NAME, "", METRICS_DIR, CONCURRENCY_VS_INFERENCE_LATENCY_PLOT_FNAME)
logger.info(f"Plot saved to s3://{BUCKET_NAME}/{METRICS_DIR}/{CONCURRENCY_VS_INFERENCE_LATENCY_PLOT_FNAME}")

In [ ]:
df_pricing = pd.DataFrame.from_dict(config['pricing'], orient='index').reset_index()
df_pricing.columns = ['instance_type', 'price_per_hour']
# fpath: str = os.path.join(METRICS_DIR, INSTANCE_PRICING_PER_HOUR_FNAME)

csv_buffer = io.StringIO()
df_pricing.to_csv(csv_buffer, index=False)
df_pricing_data = csv_buffer.getvalue()

# Write the CSV data to S3
write_to_s3(df_pricing_data, BUCKET_NAME, "", METRICS_DIR, INSTANCE_PRICING_PER_HOUR_FNAME)

df_pricing

In [ ]:
df_summary_metrics_dataset = pd.merge(df_summary_metrics_dataset, df_pricing, how='left')
logger.info(f"df_summary_metrics_dataset ----> {df_summary_metrics_dataset}")
logger.info(f"df_summary_metrics_dataset ----> {df_summary_metrics_dataset}")
df_summary_metrics_dataset['price_per_txn'] = (df_summary_metrics_dataset['price_per_hour']/60)/df_summary_metrics_dataset['transactions_per_minute']
price_per_tx_wt = config['metrics']['weights']['price_per_tx_wt']
latency_wt = config['metrics']['weights']['latenct_wt']
#df_summary_metrics_dataset['score'] = price_per_tx_wt*(1/df_summary_metrics_dataset['price_per_txn']) + latency_wt*(1/df_summary_metrics_dataset['latency_mean'])
df_summary_metrics_dataset['score'] = 0.5*(1/df_summary_metrics_dataset['price_per_txn']) + 0.5*(1/df_summary_metrics_dataset['latency_mean'])

"""
df_summary_metrics_dataset['rank'] = (df_summary_metrics_dataset.sort_values(by="score", ascending=False)
                      .groupby(['instance_type'])['concurrency']
                      .rank(method='first', ascending=False)
                   )
"""
df_summary_metrics_dataset = df_summary_metrics_dataset.sort_values(by="score", ascending=False)
file_path_df = os.path.join(METRICS_DIR, SUMMARY_METRICS_FOR_DATASET_W_SCORES_FNAME)

csv_buffer = io.StringIO()
df_summary_metrics_dataset.to_csv(file_path_df, index=False)
summary_metrics_dataset_csv = csv_buffer.getvalue()

# Write the CSV data to S3
write_to_s3(summary_metrics_dataset_csv, config['aws']['bucket'], "", METRICS_DIR, SUMMARY_METRICS_FOR_DATASET_W_SCORES_FNAME)
logger.info(f"Summary metrics dataset saved to s3://{config['aws']['bucket']}/{METRICS_DIR}/{SUMMARY_METRICS_FOR_DATASET_W_SCORES_FNAME}")

df_summary_metrics_dataset



In [ ]:
# select the best option overall and for each instance type
df_summary_metrics_dataset_overall = df_summary_metrics_dataset[df_summary_metrics_dataset.score == df_summary_metrics_dataset.score.max()]

csv_buffer = io.StringIO()
df_summary_metrics_dataset_overall.to_csv(csv_buffer, index=False)
metrics_overall_data = csv_buffer.getvalue()

# Write the CSV data to S3
write_to_s3(metrics_overall_data, BUCKET_NAME, "", METRICS_DIR, SUMMARY_METRICS_FOR_DATASET_W_SCORES_BEST_OPTION_FNAME)

df_summary_metrics_dataset_overall

In [ ]:
df_summary_metrics_dataset_overall = df_summary_metrics_dataset_overall.round(6)
df_summary_metrics_dataset_overall.to_dict(orient='records')

In [ ]:
df_summary_metrics_dataset = df_summary_metrics_dataset.dropna()
idx = df_summary_metrics_dataset.groupby(['instance_type']).score.idxmax()
logger.info(f"shape of df_summary_metrics_dataset={df_summary_metrics_dataset.shape}, idx={idx}")
df_summary_metrics_best_option_instance_type = df_summary_metrics_dataset.loc[idx]
logger.info(f"shape of df_summary_metrics_best_option_instance_type={df_summary_metrics_best_option_instance_type.shape}")

csv_buffer = io.StringIO()
df_summary_metrics_best_option_instance_type.to_csv(csv_buffer, index=False)
best_option = csv_buffer.getvalue()

# Write the CSV data to S3
write_to_s3(best_option, BUCKET_NAME, "", METRICS_DIR, SUMMARY_METRICS_FOR_DATASET_W_SCORES_BEST_OPTION_EACH_INSTANCE_TYPE_FNAME)

df_summary_metrics_best_option_instance_type


In [ ]:
df_summary_metrics_best_option_instance_type

In [ ]:
min_price_per_tx = df_summary_metrics_best_option_instance_type.price_per_txn.min()
txn_count_for_showing_cost: int = config["report"]["txn_count_for_showing_cost"]

price_tx_col_name = f"price_per_tx_{txn_count_for_showing_cost}_txn"
df_summary_metrics_best_option_instance_type[price_tx_col_name] = df_summary_metrics_best_option_instance_type.price_per_txn * txn_count_for_showing_cost
df_summary_metrics_best_option_instance_type[price_tx_col_name] = round(df_summary_metrics_best_option_instance_type[price_tx_col_name], 2)
df_summary_metrics_best_option_instance_type = df_summary_metrics_best_option_instance_type.sort_values(by=price_tx_col_name)
sns_plot = sns.barplot(df_summary_metrics_best_option_instance_type, x="instance_type", y=price_tx_col_name, hue="instance_type")
title: str = f"Comparing performance of {config['general']['model_name']} across instance types for {config['metrics']['dataset_of_interest']} dataset"
sns_plot.set(xlabel="Instance type", ylabel=f"Cost per {txn_count_for_showing_cost:,} transactions (USD)", title=title)
num_instance_types = len(df_summary_metrics_dataset.instance_type.unique())
for r in df_summary_metrics_best_option_instance_type.iterrows():
    x = r[1]['instance_type']
    if num_instance_types == 1:
        v_shift = config["report"]["v_shift_w_single_instance"]
    else:
        v_shift = config["report"]["v_shift_w_gt_one_instance"]
    
    print(f"v_shift={v_shift}")    
    y = r[1][price_tx_col_name] + v_shift
    text = f"{r[1]['transactions_per_minute']} txn/min, {r[1]['latency_mean']}s per txn"
    print(f"x={x}, y={y}, text={text}")
    sns_plot.text(x, y, text, 
       fontsize = 8,          # Size
       #fontstyle = "oblique",  # Style
       color = "red",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 

business_summary_plot_fpath: str = os.path.join(METRICS_DIR, BUSINESS_SUMMARY_PLOT_FNAME)
sns_plot.figure.savefig(buffer, format='png')
buffer.seek(0)

# Write the plot to S3
write_to_s3(buffer.getvalue(), BUCKET_NAME, "", "", business_summary_plot_fpath)
logger.info(f"Plot saved to s3://{BUCKET_NAME}/{business_summary_plot_fpath}")

## Display the plot 
sns_plot

In [ ]:
display(df_summary_metrics_best_option_instance_type)

In [ ]:
cost_csv_content_fpath = os.path.join(METRICS_DIR, SUMMARY_MODEL_ENDPOINT_COST_PER_INSTANCE)
logger.info(f"the cost information can be found in the csv file here -> {cost_csv_content_fpath}")

# Read the file from S3
try:
    cost_content = get_s3_object(BUCKET_NAME, cost_csv_content_fpath)

    # Use pandas to read the CSV content
    df_cost_metrics = pd.read_csv(io.StringIO(cost_content))
    logger.info(f"{cost_csv_content_fpath} read into dataframe of shape {df_cost_metrics.shape}")
    df_cost_metrics.head()
except Exception as e:
    logger.error(f"Error reading from S3: {e}")

# Replace NaN values with an empty string
# df_cost_metrics.fillna('', inplace=True)

df_cost_metrics.head()

# Convert df_cost_metrics to Markdown table
cost_mkdn_table = Tomark.table(df_cost_metrics.to_dict(orient='records'))

In [ ]:
BUSINESS_SUMMARY: str = """We did performance benchmarking for the `{model_name}` model on "{instance_types}" instance{plural} on multiple datasets and based on the test results the best price performance for dataset `{ds}` is provided by the `{selected_instance_type}` instance type.  {mkdn_table}

The price performance comparison for different instance types is presented below:

![Price performance comparison]({business_summary_plot_fpath})

The configuration used for these tests is available in the [`config`]({cfg_file_path}) file.

The cost to run each experiment is provided in the table below. The total cost for running all experiments is {total_cost_as_str}.

{cost_table}

"""
transposed_list = []
best_instance_type_info = df_summary_metrics_dataset_overall.to_dict(orient='records')[0]
del best_instance_type_info["score"]
for k,v in best_instance_type_info.items():
    transposed_list.append({"Information": k, "Value": v})
mkdn_table = Tomark.table(transposed_list)


plural = "s" if len(df_summary_metrics.instance_type.unique()) > 1 else ""
instance_types_md = ", ".join([f"`{it}`" for it in df_summary_metrics.instance_type.unique()])
selected_instance_type: str = df_summary_metrics_dataset_overall.to_dict(orient='records')[0]['instance_type']
ds: str = config['metrics']['dataset_of_interest']

business_summary: str = BUSINESS_SUMMARY.format(model_name=config['general']['model_name'],
                                              instance_types=instance_types_md,
                                              plural=plural,
                                              ds=ds,
                                              selected_instance_type=selected_instance_type,
                                              mkdn_table="\n" + mkdn_table,
                                              cfg_file_path=os.path.basename(CONFIG_FILE),
                                              business_summary_plot_fpath=BUSINESS_SUMMARY_PLOT_FNAME,
                                              cost_table=cost_mkdn_table,
                                              total_cost_as_str=f"${df_cost_metrics.cost.sum():.2f}"
                                              )
business_summary

In [ ]:
from typing import Optional
dttm = str(datetime.utcnow())
datasets_used = ", ".join([f"`{d}`" for d in config['s3_read_data']['source_data_files']])
overall_results_md = OVERALL_RESULTS_MD.format(dttm=dttm,
                                               business_summary=business_summary,
                                               datasets=datasets_used)

results_group_cols: List[str] = ['instance_type', 'payload_file']
result_rows: List[str] = []
for row in df_summary_metrics[results_group_cols].drop_duplicates().iterrows():
    instance_type = row[1]['instance_type']
    dataset = row[1]['payload_file']
    df_summary_metrics_nz_subset = df_summary_metrics_nz[(df_summary_metrics_nz.instance_type == instance_type) &
                                                          (df_summary_metrics_nz.payload_file == dataset) &
                                                           (df_summary_metrics_nz.latency_mean <= LATENCY_BUDGET)]
    num_results = df_summary_metrics_nz_subset.shape[0]
    result_row: Optional[str] = None
    if num_results > 0:
        logger.info(f"there are {num_results} options to choose the best option from for instance_type={instance_type}, dataset={dataset}")
        df_summary_metrics_nz_subset_selected = df_summary_metrics_nz_subset[df_summary_metrics_nz_subset.concurrency == df_summary_metrics_nz_subset.concurrency.max()]
        best = df_summary_metrics_nz_subset_selected.to_dict(orient='records')[0]
        # logger.info(best)
        result_desc = RESULT_DESC.format(latency_budget=LATENCY_BUDGET,
                           instance_type=best['instance_type'],
                           dataset=dataset,
                           concurrency=best['concurrency'],
                           latency_mean=best['latency_mean'],
                           prompt_size=int(best['prompt_token_count_mean']),
                           completion_size=int(best['completion_token_count_mean']),
                           tpm=int(best['transactions_per_minute']))     
        
        # logger.info(result_desc)
    else:
        logger.info(f"there are NO options to choose from for instance_type={instance_type}, dataset={dataset}")
        result_desc = RESULT_FAILURE_DESC.format(latency_budget=LATENCY_BUDGET,
                           instance_type=best['instance_type'],
                           dataset=dataset)
    result_row: str = RESULT_ROW.format(instance_type=best['instance_type'],
                                        dataset=dataset,
                                        desc=result_desc)
    result_rows.append(result_row)
        
    
    #logger.info(f"instance_type={row[0]}, payload_file={row[1]}")
overall_results_md += "\n".join(result_rows)

OVERALL_RESULTS_PLOTS_MD: str = """

## Plots

The following plots provide insights into the results from the different experiments run.

![{plot1_text}]({plot1_fname})

![{plot2_text}]({plot2_fname})

![{plot3_text}]({plot3_fname})
"""

overall_results_plots_md: str = OVERALL_RESULTS_PLOTS_MD.format(plot1_text=ERROR_RATES_PLOT_TEXT, 
                                                                plot1_fname=ERROR_RATES_PLOT_FNAME,
                                                                plot2_text=TOKENS_VS_LATENCY_PLOT_TEXT, 
                                                                plot2_fname=TOKENS_VS_LATENCY_PLOT_FNAME,
                                                                plot3_text=CONCURRENCY_VS_INFERENCE_LATENCY_PLOT_TEXT, 
                                                                plot3_fname=CONCURRENCY_VS_INFERENCE_LATENCY_PLOT_FNAME)

overall_results_md += overall_results_plots_md

fpath: str = os.path.join(METRICS_DIR, RESULTS_DESC_MD_FNAME)
logger.info(f"writing final markdown to {METRICS_DIR}")
Path(fpath).write_text(overall_results_md)
logger.info(overall_results_md)

# Write the CSV data to S3
write_to_s3(overall_results_md, BUCKET_NAME, "", METRICS_DIR, RESULTS_DESC_MD_FNAME)
logger.info(f"results.md file saved to to s3://{BUCKET_NAME}/{METRICS_DIR}/{RESULTS_DESC_MD_FNAME}")

In [ ]:
#save all the metrics and report files locally
os.makedirs(RESULTS_DIR, exist_ok=True)
logger.info(f"going to download all metrics and reports from s3 into {RESULTS_DIR} directory")
download_multiple_files_from_s3(BUCKET_NAME, METRICS_DIR, RESULTS_DIR)
import glob
result_files = glob.glob(os.path.join(RESULTS_DIR, "**"), recursive=True)
logger.info("\n".join([f for f in result_files]))

In [14]:
"""
Get information about models in Amazon SageMaker JumpStart.
Tested with sagemaker SDK version 2.203.0.
"""

## View the full list of models available through JumpStart here: https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html

import pandas as pd
from typing import Dict, List
from sagemaker import get_execution_role
from sagemaker.jumpstart.model import JumpStartModel

# global constants
try:
    ROLE_ARN: str = get_execution_role()
except Exception as e:
    # if not running this code on SageMaker, set the IAM role ARN to use here
    ROLE_ARN: str = "arn:aws:iam::<your-account-id>:role/<role-name>"

# models of interest
JS_MODEL_ID_LIST: List = ['meta-textgenerationneuron-llama-2-70b-f']

## Add pricing as needed, using the instances

## These are per-hour prices for us-east-1 below, 
## PLEASE CONSULT Amazon SageMaker Pricing page https://aws.amazon.com/sagemaker/pricing/ for up to date information.
PRICING_PER_INSTANCE: Dict = {
    "ml.g5.2xlarge": 1.515,
    "ml.p4de.24xlarge": 40.966,
    "ml.g5.12xlarge": 7.09,
    "ml.g5.24xlarge": 10.18,
    "ml.g5.48xlarge": 20.36,
    "ml.inf2.24xlarge": 7.79,
    "ml.inf2.48xlarge": 15.58,
    "ml.p4d.24xlarge": 37.688,
    "ml.p3.2xlarge": 3.825,
    "ml.m5.xlarge": 0.23
}
## convert the pricing information into a DF
pricing_df = pd.DataFrame(list(PRICING_PER_INSTANCE.items()), columns=['instance_type', 'price_per_hour'])

JS_MODEL_INFO_CSV_FNAME: str = "jumpstart_models.csv"
print(f"SageMaker execution role ARN being used -> {ROLE_ARN}")

def get_model_info(model_id: str, model_version: str = "*") -> Dict:
    """
    Get information about the model that Amazon SageMaker JumpStart would use to deploy it.
    """
    try:
        # Attempt to create a JumpStartModel instance if not then catch the non jumpstart model
        my_model: JumpStartModel = JumpStartModel(model_id=model_id, model_version=model_version, role=ROLE_ARN)
        # Initialize model configurations
        model_config = vars(my_model)
        print(json.dumps(model_config, indent=2, default=str))
        instance_type = model_config.get('instance_type')
        
        print(f"model_id={model_id}, model_version={model_version}, recommended instance_type={instance_type}")
        return dict(model_id=model_id, model_version=model_version, instance_type=instance_type, model_config=model_config)         
    except Exception as e:
        # Handle the case where the model is not part of JumpStart
        print(f"Model Id {model_id} is currently not supported by JumpStart")
        return dict(model_id=model_id, model_version=model_version, instance_type=None, model_config=None)
    
model_info = list(map(get_model_info, JS_MODEL_ID_LIST))

# Load the list into a dataframe
df = pd.DataFrame(model_info)
df = df.sort_values(by="model_id")
print(df)

# Merge df with pricing_df on 'instance_type'
merged_df = pd.merge(df, pricing_df, on='instance_type', how='left')

first_few_cols = ['model_id', 'model_version', 'instance_type', 'price_per_hour']
col_order = first_few_cols +  [col for col in merged_df.columns if col not in first_few_cols]
final_df = merged_df[col_order]

# Display the final DataFrame
print(final_df)

## display it in the final csv
#final_df.to_csv(JS_MODEL_INFO_CSV_FNAME, index=False)
#print(f"CSV file with merged model info and pricing of shape {final_df.shape} saved at: {JS_MODEL_INFO_CSV_FNAME}")

No instance type selected for inference hosting endpoint. Defaulting to ml.inf2.48xlarge.
[2024-04-02 16:35:44,088] p26924 {model.py:193} INFO - No instance type selected for inference hosting endpoint. Defaulting to ml.inf2.48xlarge.


SageMaker execution role ARN being used -> arn:aws:iam::015469603702:role/fmbench-role
{
  "_model_data_is_set": false,
  "orig_predictor_cls": null,
  "model_id": "meta-textgenerationneuron-llama-2-70b-f",
  "model_version": "*",
  "instance_type": "ml.inf2.48xlarge",
  "resources": "ResourceRequirements: {'requests': {'num_accelerators': 12, 'memory': 393216}, 'limits': {}, 'num_accelerators': 12, 'min_memory': 393216, 'copy_count': 1}",
  "tolerate_vulnerable_model": false,
  "tolerate_deprecated_model": false,
  "region": "us-east-1",
  "sagemaker_session": "<sagemaker.session.Session object at 0x7fc0a650d590>",
  "model_data": {
    "S3DataSource": {
      "S3Uri": "s3://jumpstart-private-cache-prod-us-east-1/meta-textgenerationneuron/meta-textgenerationneuron-llama-2-70b-f/artifacts/inference-prepack/v1.0.0/",
      "S3DataType": "S3Prefix",
      "CompressionType": "None"
    }
  },
  "image_uri": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:1.